In [1]:
import pandas as pd
import numpy as np
from kite_trade import *
from enctoken import get_kite
kite = get_kite()
import warnings
warnings.filterwarnings(action = 'ignore')
import time
from IPython.display import clear_output
from joblib import Parallel, delayed
from tqdm import tqdm
import xlwings
import datetime
import mibian
import swifter
from joblib import Parallel, delayed

## option selling stretagy 1

In [2]:
def buy_sell_order(strike_price = None,ce_pe = None, price = None, trans_type = None,order_type = "MARKET",quantity = 15 ):
    print(strike_price, ce_pe, price,trans_type, order_type)
    order = kite.place_order(
        variety = kite.VARIETY_REGULAR,
        exchange = kite.EXCHANGE_NFO,
        tradingsymbol = strike_price,
        transaction_type = trans_type,
        quantity = quantity,
        product = kite.PRODUCT_MIS,
        order_type = order_type,
        price=price,
        validity=kite.VALIDITY_DAY,
        disclosed_quantity=None,
        trigger_price=None,
        squareoff=None,
        stoploss=None,
        trailing_stoploss=None,
        # tag="TradeviaPython",
    )
    return order

def get_stike_of_price(df, price):
    # find the closest value of a value
    df["diff"] = abs(df["last_price"] - price)
    ce_side = (
        df.sort_values("diff")
        .query('instrument_type =="CE"')
        .reset_index(drop=True)
        .head(1)["strike_fr_ltp"][0]
    )
    pe_side = (
        df.sort_values("diff")
        .query('instrument_type =="PE"')
        .reset_index(drop=True)
        .head(1)["strike_fr_ltp"][0]
    )
    return ce_side, pe_side


def get_atm_stike():
    bn_ltp = kite.ltp("NSE:NIFTY BANK")["NSE:NIFTY BANK"]["last_price"]
    if bn_ltp % 100 > 50:
        atm_stike = bn_ltp - bn_ltp % 100 + 100
    else:
        atm_stike = bn_ltp - bn_ltp % 100
    return int(atm_stike)


In [3]:
expiry_day = 'BANKNIFTY23SEP'
lower_price = 40000
upper_price = 50000

inst = pd.DataFrame(kite.instruments("NFO"))
expiry_march = (
    inst[(inst.name == "BANKNIFTY") & (inst.tradingsymbol.str.contains(expiry_day))]
    # .query("(strike >= @lower_price )&( strike <= @upper_price)")
    .assign(strike_type=lambda x: x["strike"].astype(int).astype(str) + x["instrument_type"])
)
expiry_march = expiry_march[['instrument_token','tradingsymbol','last_price','instrument_type','strike_type']]
print('shape of expiry data:',expiry_march.shape)
expiry_march['strike_fr_ltp'] = f"NFO:{expiry_day}" + expiry_march.strike_type
bn_ltp = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
expiry_march = (
    expiry_march.drop("last_price", axis=1).merge(
        pd.DataFrame(kite.ltp(expiry_march.strike_fr_ltp.unique()))
        .T.reset_index()
        .rename(columns=({"index": "strike_fr_ltp"}))
        .drop("instrument_token", axis=1),
        on="strike_fr_ltp",
    )
)

ce_buy_5, pe_buy_5 = get_stike_of_price(expiry_march,5)
ce_sell_80, pe_sell_80 = get_stike_of_price(expiry_march,80)
atm_stike = get_atm_stike()
ce_sell_atm = f"NFO:{expiry_day}"+str(atm_stike)+"CE"
pe_sell_atm = f"NFO:{expiry_day}"+str(atm_stike)+"PE"
print("ltp :",[str(k) +"  "+ str(v['last_price'])  for k, v in kite.ltp([ce_sell_atm,pe_sell_atm,ce_sell_80,pe_sell_80]).items()])

shape of expiry data: (189, 5)
ltp : ['NFO:BANKNIFTY23SEP44200PE  70.9', 'NFO:BANKNIFTY23SEP44500CE  162.5', 'NFO:BANKNIFTY23SEP44500PE  181.55', 'NFO:BANKNIFTY23SEP44700CE  83.8']


In [4]:

# buy 1 lot for 5 rs
order_type="MARKET",
quantity=15,

ord1 = buy_sell_order(
    strike_price=ce_buy_5[4:],
    trans_type=kite.TRANSACTION_TYPE_BUY,
    quantity=30
)

ord2 = buy_sell_order(
    strike_price=pe_buy_5[4:],
    trans_type=kite.TRANSACTION_TYPE_BUY,
    quantity=30
)

# sell 1 lot for 80 rs
time.sleep(3)
ord3 = buy_sell_order(
    strike_price=ce_sell_80[4:],
    trans_type=kite.TRANSACTION_TYPE_SELL,
)

ord4 = buy_sell_order(
    strike_price=pe_sell_80[4:],
    trans_type=kite.TRANSACTION_TYPE_SELL,
)

# sell 1 lot for atm
time.sleep(5)
ord5 = buy_sell_order(
    strike_price=ce_sell_atm[4:],
    trans_type=kite.TRANSACTION_TYPE_SELL,

)

ord6 = buy_sell_order(
    strike_price=pe_sell_atm[4:],
    trans_type=kite.TRANSACTION_TYPE_SELL,
)

BANKNIFTY23SEP45800CE None None BUY MARKET
BANKNIFTY23SEP43400PE None None BUY MARKET
BANKNIFTY23SEP44700CE None None SELL MARKET
BANKNIFTY23SEP44200PE None None SELL MARKET
BANKNIFTY23SEP44500CE None None SELL MARKET
BANKNIFTY23SEP44500PE None None SELL MARKET


In [ ]:
ce_80_flag = False
pe_80_flag = True
ce_atm_flag = False
pe_atm_flag = True

entry_price_dict = kite.ltp([ce_sell_80,pe_sell_80,ce_sell_atm,pe_sell_atm])

In [ ]:
# flags 
while True:
    ltp_price = kite.ltp([ce_sell_80,pe_sell_80,ce_sell_atm,pe_sell_atm])
    if ltp_price[ce_sell_atm]['last_price'] > entry_price_dict[ce_sell_atm]['last_price']*1.1 and not ce_atm_flag:
        print("sq-off CE ATM: ",ce_sell_atm)
        buy_sell_order(
            strike_price=ce_sell_atm[4:],
            trans_type=kite.TRANSACTION_TYPE_BUY
        )
        ce_atm_flag = True

    if ltp_price[pe_sell_atm]['last_price'] > entry_price_dict[pe_sell_atm]['last_price']*1.1 and not pe_atm_flag:
        print("sq-off PE ATM: ",pe_sell_atm)
        buy_sell_order(
            strike_price=pe_sell_atm[4:],
            trans_type=kite.TRANSACTION_TYPE_BUY
        )
        pe_atm_flag = True
        
    if ltp_price[ce_sell_80]['last_price'] > entry_price_dict[ce_sell_80]['last_price']*1.3 and not ce_80_flag:
        print("sq-off CE 80: ",ce_sell_80)
        buy_sell_order(
            strike_price=ce_sell_80[4:],
            trans_type=kite.TRANSACTION_TYPE_BUY
        )
        ce_80_flag = True

    if ltp_price[pe_sell_80]['last_price'] > entry_price_dict[pe_sell_80]['last_price']*1.3 and not pe_80_flag:
        print("sq-off PE 80: ",pe_sell_80)
        buy_sell_order(
            strike_price=pe_sell_80[4:],
            trans_type=kite.TRANSACTION_TYPE_BUY
        )
        pe_80_flag = True


    # check can we reenter the trade
    if ce_atm_flag and ltp_price[ce_sell_atm]['last_price'] <= entry_price_dict[ce_sell_atm]['last_price']:
        print('renter CE ATM: ',ce_sell_atm)
        buy_sell_order(
            strike_price=ce_sell_atm[4:],
            trans_type= kite.TRANSACTION_TYPE_SELL
        )
        ce_atm_flag = False

    if pe_atm_flag and ltp_price[pe_sell_atm]['last_price'] <= entry_price_dict[pe_sell_atm]['last_price']:
        print('renter PE ATM: ',pe_sell_atm)
        buy_sell_order(
            strike_price=pe_sell_atm[4:],
            trans_type=kite.TRANSACTION_TYPE_SELL
        )
        pe_atm_flag = False
        

    if ce_80_flag and ltp_price[ce_sell_80]['last_price'] <= entry_price_dict[ce_sell_80]['last_price']:
        print('renter CE 80: ',ce_sell_80)
        buy_sell_order(
            strike_price=ce_sell_80[4:],
            trans_type=kite.TRANSACTION_TYPE_SELL
        )
        ce_80_flag = False

    if pe_80_flag and ltp_price[pe_sell_80]['last_price'] <= entry_price_dict[pe_sell_80]['last_price']:
        print('renter PE 80: ',pe_sell_80)
        buy_sell_order(
            strike_price=pe_sell_80[4:],
            trans_type=kite.TRANSACTION_TYPE_SELL
        )
        pe_80_flag = False
    
    #exit at 3.20 pm
    current_time = datetime.datetime.now().time()
    if current_time.hour == 3 and current_time.minute >= 20:
        print("exiting all positions")
        if not ce_atm_flag:
            buy_sell_order(
            strike_price=ce_sell_atm[4:],
            trans_type=kite.TRANSACTION_TYPE_BUY
            )
        if not pe_atm_flag:
            buy_sell_order(
            strike_price=pe_sell_atm[4:],
            trans_type=kite.TRANSACTION_TYPE_BUY
            )
        if not ce_80_flag:
            buy_sell_order(
                strike_price= ce_sell_80[4:],
                trans_type=kite.TRANSACTION_TYPE_BUY
            )
        if not pe_80_flag:
            buy_sell_order(
                strike_price= pe_sell_80[4:],
                trans_type=kite.TRANSACTION_TYPE_BUY
            )

        time.sleep(10)

        buy_sell_order(
        strike_price=ce_buy_5[4:],
        trans_type=kite.TRANSACTION_TYPE_SELL,
        quantity=30
         )
        buy_sell_order(
            strike_price=pe_buy_5[4:],
            trans_type=kite.TRANSACTION_TYPE_SELL,
            quantity=30
        )
        break        
    time.sleep(10)

In [80]:
# pd.DataFrame(kite.positions()['net']).T

In [81]:
# pd.DataFrame(kite.orders()).T

In [ ]:
# exit at 3.20  -> Done 
# 1. overall loss 1% to square of all position 
# 2. renter 3 times only 
# 3. message notification for all entry and exit 
# 4. show relevnet information 
#     price at which we enter the trade
#     time of entry
# logs every excution in excel file
# code to restore the state of running program






## Backtest option selling stretegy 1 

In [ ]:
#at the start of day at 9:20, place all 6 orders
# iterate over each row and check the condition of sl or rentry 
# at each square of position calcalute the profit and loss
# at 3.20 square of all position and compute the final lose and profit

In [ ]:
def date_process(df):
    df = df.sort_values('date')
    df['date_only'] = pd.to_datetime(df.date.dt.date)
    df.set_index('date', inplace=True)
    df = df.between_time('09:20', '03:20')
    df.index = df.index.tz_localize(None)
    return df 
    

def get_stike_of_price(df, price):
    # find the closest value of a value
    df["diff"] = abs(df["last_price"] - price)
    ce_side = (
        df.sort_values("diff")
        .query('instrument_type =="CE"')
        .reset_index(drop=True)
        .head(1)["strike_type"][0]
    )
    pe_side = (
        df.sort_values("diff")
        .query('instrument_type =="PE"')
        .reset_index(drop=True)
        .head(1)["strike_type"][0]
    )
    return ce_side, pe_side


def get_atm_stike(bn_ltp = None):
    if bn_ltp is None:
        bn_ltp = kite.ltp("NSE:NIFTY BANK")["NSE:NIFTY BANK"]["last_price"]
        
    if bn_ltp % 100 > 50:
        atm_stike = bn_ltp - bn_ltp % 100 + 100
    else:
        atm_stike = bn_ltp - bn_ltp % 100
    return int(atm_stike)


def get_strike_ltp(df, strike):
    return df.between_time('09:20', '09:20').query('strike_type == @strike')['last_price'].values[0]
    
# def get_current_price(df,)

In [ ]:
start_dt = '2023-05-01'
end_dt = '2023-05-22'
time_frame = 'minute'
bn_token  = 260105

df_strike_data = pd.read_parquet('../data/expiry_18may_1min.parquet')
df_bn_hist_data = (
        pd.DataFrame(
            kite.historical_data(
                bn_token,
                from_date=start_dt,
                to_date=end_dt,
                interval=time_frame,
                continuous=False,
                oi=True,
            )
        )
        .assign(strike_type="BANKNIFTY")
    )
df_strike_data = date_process(df_strike_data)
df_bn_hist_data = date_process(df_bn_hist_data)
df_strike_data = df_strike_data.query('date_only >= "2023-05-11"')

In [ ]:
import pandas_flavor as pf
@pf.register_dataframe_method
def get_last_price(df, strike):
    return df.between_time('09:20', '09:20').query('strike_type == @strike')['last_price'].values[0]

In [ ]:
# p_and_l = 0
# ce_80_flag = False
# pe_80_flag = False
# ce_atm_flag = False
# pe_atm_flag = False
# ce_atm_rentry_price = 0
# pe_atm_rentry_price = 0
# ce_80_rentry_price = 0
# pe_80_rentry_price = 0


for dts in df_strike_data.date_only.unique():
    p_and_l = 0
    ce_80_flag = False
    pe_80_flag = False
    ce_atm_flag = False
    pe_atm_flag = False
    ce_atm_rentry_price = 0
    pe_atm_rentry_price = 0
    ce_80_rentry_price = 0
    pe_80_rentry_price = 0
    
    print(dts)
    df_one_day = df_strike_data.query('date_only == @dts')
    df_bn_day = df_bn_hist_data.query('date_only == @dts')

    df_one_day['instrument_type'] = df_one_day.strike_type.str[-2:]
    df_one_day['last_price'] = df_one_day['close']

    # getting strike value
    bn_atm_strike = get_atm_stike(df_bn_day.between_time('09:20', '09:20')['close'].values[0])
    ce_buy_5, pe_buy_5 = get_stike_of_price(df_one_day.between_time('09:20', '09:20'), 5)
    ce_sell_80, pe_sell_80 = get_stike_of_price(df_one_day.between_time('09:20', '09:20'), 80)
    ce_sell_atm = f"{bn_atm_strike}CE"
    pe_sell_atm = f"{bn_atm_strike}PE"

    # getting strike price at 9:20 
    ce_atm_price = get_strike_ltp(df_one_day, ce_sell_atm)
    pe_atm_price = get_strike_ltp(df_one_day, pe_sell_atm)
    ce_sell_80_price = get_strike_ltp(df_one_day, ce_sell_80)
    pe_sell_80_price = get_strike_ltp(df_one_day, pe_sell_80)

    for dt_time in df_one_day.index.unique():
        df_temp = df_one_day.reset_index().query('date == @dt_time')
        # at 9:20 what is the bn price and get all the strike price to place the order
        if df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0] > ce_atm_price * 1.1 and not ce_atm_flag:
            print("square off CE atm: ",df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0],ce_atm_price)
            p_and_l -= df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0] - ce_atm_price 
            ce_atm_flag = True
        
        if df_temp.query("strike_type == @pe_sell_atm")['last_price'].values[0] > pe_atm_price * 1.1 and not pe_atm_flag:
            print("square off PE atm: ",df_temp.query("strike_type == @pe_sell_atm")['last_price'].values[0],pe_atm_price)
            p_and_l -= df_temp.query("strike_type == @pe_sell_atm")['last_price'].values[0] - pe_atm_price
            pe_atm_flag = True
        
        if df_temp.query("strike_type == @ce_sell_80")['last_price'].values[0] > ce_sell_80_price * 1.3 and not ce_80_flag:
            print("square off CE 80: ",df_temp.query("strike_type == @ce_sell_80")['last_price'].values[0],ce_sell_80_price)
            p_and_l -= df_temp.query("strike_type == @ce_sell_80")['last_price'].values[0] - ce_sell_80_price
            ce_80_flag = True

        if df_temp.query("strike_type == @pe_sell_80")['last_price'].values[0] > pe_sell_80_price * 1.3 and not pe_80_flag:
            print("square off PE 80: ",df_temp.query("strike_type == @pe_sell_80")['last_price'].values[0],pe_sell_80_price)
            p_and_l -= df_temp.query("strike_type == @pe_sell_80")['last_price'].values[0] - pe_sell_80_price
            pe_80_flag = True

        # renetry condition
        if ce_atm_flag and df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0] <= ce_atm_price:
            print("re-entry CE atm: ",df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0],ce_atm_price)
            ce_atm_flag = False
            ce_atm_rentry_price = df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0]
        
        if pe_atm_flag and df_temp.query("strike_type == @pe_sell_atm")['last_price'].values[0] <= pe_atm_price:
            print("re-entry PE atm: ",df_temp.query("strike_type == @pe_sell_atm")['last_price'].values[0],pe_atm_price)
            pe_atm_flag = False
            pe_atm_rentry_price = df_temp.query("strike_type == @pe_sell_atm")['last_price'].values[0]

        if ce_80_flag and df_temp.query("strike_type == @ce_sell_80")['last_price'].values[0] <= ce_sell_80_price:
            print("re-entry CE 80: ",df_temp.query("strike_type == @ce_sell_80")['last_price'].values[0],ce_sell_80_price)
            ce_80_flag = False
            ce_80_rentry_price = df_temp.query("strike_type == @ce_sell_80")['last_price'].values[0]
        
        if pe_80_flag and df_temp.query("strike_type == @pe_sell_80")['last_price'].values[0] <= pe_sell_80_price:
            print("re-entry PE 80: ",df_temp.query("strike_type == @pe_sell_80")['last_price'].values[0],pe_sell_80_price)
            pe_80_flag = False
            pe_80_rentry_price = df_temp.query("strike_type == @pe_sell_80")['last_price'].values[0]

    if not ce_atm_flag:
        if ce_atm_rentry_price > 0:
            print(ce_atm_rentry_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @ce_sell_atm')['last_price'].values[0])
        else:
            print(ce_atm_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @ce_sell_atm')['last_price'].values[0])
    
    if not pe_atm_flag:
        if pe_atm_rentry_price > 0:
            print(pe_atm_rentry_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @pe_sell_atm')['last_price'].values[0])
        else:
            print(pe_atm_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @pe_sell_atm')['last_price'].values[0])
    
    if not ce_80_flag:
        if ce_80_rentry_price > 0:
            print(ce_80_rentry_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @ce_sell_80')['last_price'].values[0])
        else:
            print(ce_sell_80_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @ce_sell_80')['last_price'].values[0])
    
    if not pe_80_flag:
        if pe_80_rentry_price > 0:
            print(pe_80_rentry_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @pe_sell_80')['last_price'].values[0])
        else:
            print(pe_sell_80_price - df_one_day.between_time('15:20', '15:20').query('strike_type == @pe_sell_80')['last_price'].values[0])

    print(p_and_l)




2023-05-11T00:00:00.000000000
square off PE atm:  391.55 355.85
re-entry PE atm:  352.0 355.85
square off CE atm:  325.4 292.9
square off CE 80:  114.0 85.2
re-entry CE 80:  85.1 85.2
re-entry CE atm:  291.5 292.9
square off CE atm:  325.1 292.9
square off CE 80:  114.05 85.2
re-entry CE atm:  284.3 292.9
re-entry CE 80:  81.3 85.2
0.4000000000000341
25.75
4.25
13.599999999999994
-158.05000000000004
2023-05-12T00:00:00.000000000
square off PE atm:  367.8 332.1
re-entry PE atm:  330.5 332.1
square off CE atm:  317.45 287.4
re-entry CE atm:  282.2 287.4
square off CE atm:  326.1 287.4
square off CE 80:  105.75 75.6
137.15
30.699999999999996
-134.60000000000005
2023-05-15T00:00:00.000000000
square off CE atm:  266.15 239.4
re-entry CE atm:  230.5 239.4
square off CE atm:  278.95 239.4
square off CE 80:  93.55 70.6
162.70000000000002
39.199999999999996
-89.24999999999996
2023-05-16T00:00:00.000000000
square off PE atm:  253.2 227.5
re-entry PE atm:  222.3 227.5
square off CE atm:  291.4 26

# option sellling stretagy delta

In [2]:
def date_process(df):
    df = df.sort_values('date')
    df['date_only'] = pd.to_datetime(df.date.dt.date)
    df.set_index('date', inplace=True)
    df = df.between_time('09:20', '15:20')
    df.index = df.index.tz_localize(None)
    return df 
    

def get_stike_of_price(df, price):
    # find the closest value of a value
    df["diff"] = abs(df["last_price"] - price)
    ce_side = (
        df.sort_values("diff")
        .query('instrument_type =="CE"')
        .reset_index(drop=True)
        .head(1)["strike_type"][0]
    )
    pe_side = (
        df.sort_values("diff")
        .query('instrument_type =="PE"')
        .reset_index(drop=True)
        .head(1)["strike_type"][0]
    )
    return ce_side, pe_side


def get_atm_stike(bn_ltp = None):
    if bn_ltp is None:
        bn_ltp = kite.ltp("NSE:NIFTY BANK")["NSE:NIFTY BANK"]["last_price"]
        
    if bn_ltp % 100 > 50:
        atm_stike = bn_ltp - bn_ltp % 100 + 100
    else:
        atm_stike = bn_ltp - bn_ltp % 100
    return int(atm_stike)


def get_strike_ltp(df, strike):
    return df.between_time('09:20', '09:20').query('strike_type == @strike')['last_price'].values[0]

# parllel for loop


# def get_current_price(df,)

In [3]:
start_dt = '2023-09-01'
end_dt = '2023-09-22'
time_frame = 'minute'
bn_token  = 260105

df_strike_data = pd.read_parquet('../data/expiry_20sept_min.parquet')
df_bn_hist_data = (
        pd.DataFrame(
            kite.historical_data(
                bn_token,
                from_date=start_dt,
                to_date=end_dt,
                interval=time_frame,
                continuous=False,
                oi=True,
            )
        )
        .assign(strike_type="BANKNIFTY")
    )
df_strike_data = date_process(df_strike_data)
df_bn_hist_data = date_process(df_bn_hist_data)
df_strike_data = df_strike_data.query('date_only >= "2023-09-14"')

In [4]:
df_strike_data["expiry_date"] = pd.to_datetime("2023-09-20")
df_strike_data = df_strike_data.reset_index().merge(
    df_bn_hist_data.reset_index()[["date", "close"]].rename(
        columns={"close": "bn_close"}
    ),
    on="date",
)
df_strike_data["strike"] = df_strike_data["strike_type"].str[:-2].astype(int)
df_strike_data["days_to_expiry"] = (
    df_strike_data["expiry_date"] - df_strike_data["date_only"]
).dt.days


In [5]:
# df_strike_data.drop("cval", axis=1, inplace=True)
# df_strike_data.to_parquet('../../../strike_data.parquet')

In [6]:
df_strike_data.head()

,date,strike_type,open,high,low,close,volume,oi,date_only,expiry_date,bn_close,strike,days_to_expiry
0,2023-09-14 09:20:00,44900CE,1219.25,1228.00,1211.15,1219.30,465.0,435.0,2023-09-14,2023-09-20,46065.7,44900,6
1,2023-09-14 09:20:00,46900CE,52.30,54.00,49.35,52.00,79875.0,188280.0,2023-09-14,2023-09-20,46065.7,46900,6
2,2023-09-14 09:20:00,41900PE,6.60,6.65,6.30,6.40,7935.0,15105.0,2023-09-14,2023-09-20,46065.7,41900,6
3,2023-09-14 09:20:00,46000PE,252.85,267.95,252.85,260.95,512685.0,1012470.0,2023-09-14,2023-09-20,46065.7,46000,6
4,2023-09-14 09:20:00,38000PE,3.70,3.70,3.55,3.60,29085.0,564660.0,2023-09-14,2023-09-20,46065.7,38000,6


In [35]:
interest_rate = 6.86

for i in range(7,0,-1):
    days_to_expiry = i
    print(i)
    underlying_price = 44603
    call_strike = 44700
    call_price= 169.95
    print(underlying_price, call_strike, days_to_expiry)
    civ =  mibian.BS(
        [underlying_price, call_strike, interest_rate, days_to_expiry], callPrice=call_price
    )
    cval = mibian.BS([underlying_price, call_strike, interest_rate, days_to_expiry],volatility = civ.impliedVolatility ,callPrice= call_price)
    print(civ.impliedVolatility,cval.callDelta,cval.callTheta, cval.vega,cval.gamma)


    underlying_price = 44603
    put_strike = 44700
    put_price = 218.55
    piv = mibian.BS([underlying_price, put_strike, interest_rate, days_to_expiry], putPrice= put_price)
    pval = mibian.BS([underlying_price, put_strike, interest_rate, days_to_expiry],volatility = piv.impliedVolatility ,putPrice= put_price)
    print(piv.impliedVolatility,pval.callDelta-1,cval.callTheta, cval.vega,cval.gamma)

7
44603 44700 7
7.644176483154297 0.4698525999319113 -17.323226276051077 24.57165012342564 0.0008425007599831293
8.066177368164062 -0.5283464745079594 -17.323226276051077 24.57165012342564 0.0008425007599831293
6
44603 44700 6
8.42738151550293 0.4636327727913139 -19.809964532584335 22.719241200959615 0.0008243556907884069
8.514404296875 -0.5359524420796138 -19.809964532584335 22.719241200959615 0.0008243556907884069
5
44603 44700 5
9.416580200195312 0.4576592568322392 -23.305302463254737 20.708922703534935 0.0008069730236138283
9.106636047363281 -0.5439224010718273 -23.305302463254737 20.708922703534935 0.0008069730236138283
4
44603 44700 4
10.732173919677734 0.4519139723456449 -28.564045413146392 18.492154938194204 0.000790322878063472
9.931087493896484 -0.5522887704370666 -28.564045413146392 18.492154938194204 0.000790322878063472
3
44603 44700 3
12.625694274902344 0.4463830064295502 -37.34929887264048 15.986089660856528 0.0007743377720500006
11.172294616699219 -0.5610946554100236 -3

In [52]:
interest_rate = 25
def process_ce(c):
    underlying_price = c.bn_close
    call_strike = c.strike
    days_to_expiry = c.days_to_expiry
    call_price= c.close
    print(underlying_price, call_strike, days_to_expiry)
    civ =  mibian.BS(
        [underlying_price, call_strike, interest_rate, days_to_expiry], callPrice=call_price
    )
    cval = mibian.BS([underlying_price, call_strike, interest_rate, days_to_expiry],volatility = civ.impliedVolatility ,callPrice= call_price)
    return cval.callDelta

def process_pe(c):
    underlying_price = c.bn_close
    put_strike = c.strike
    days_to_expiry = c.days_to_expiry
    put_price = c.close
    piv = mibian.BS([underlying_price, put_strike, interest_rate, days_to_expiry], putPrice= put_price)
    pval = mibian.BS([underlying_price, put_strike, interest_rate, days_to_expiry],volatility = piv.impliedVolatility ,putPrice= put_price)
    return pval.callDelta
    
    
def get_delta_25(df, d = .25):
    df['diff'] = df['delta'] - d
    return df.query('diff > 0').sort_values('diff').strike_type.head(1).values[0]
    

In [54]:
for dts in df_strike_data.date_only.unique():
    print(dts)
    df_strike_day = df_strike_data.query('date_only == @dts')
    df_strike_day["side"] = df_strike_day['strike_type'].str[-2:]
    df_strike_day = df_strike_day.set_index('date').between_time('9:45',"13:20")
    df_strike_day = df_strike_day.query('days_to_expiry != 0')
    
    df_delta = df_strike_day.between_time('9:45',"09:45")
    df_delta_ce = df_delta.query('side == "CE"')
    df_delta_pe = df_delta.query('side == "PE"')
    print(df_delta_ce.shape, df_delta_pe.shape)
    result_ce = Parallel(n_jobs=8)(delayed(process_ce)(c) for r,c in df_delta_ce.iterrows())
    df_delta_ce['delta'] = result_ce
    result_pe = Parallel(n_jobs=8)(delayed(process_pe)(c) for r,c in df_delta_pe.iterrows())
    df_delta_pe['delta'] = result_pe
    strike_ce_delta_25 = get_delta_25(df_delta_ce)
    strike_pe_delta_25 = get_delta_25(df_delta_pe)
    break

2023-09-14T00:00:00.000000000
(61, 13) (72, 13)


In [64]:
df_delta_pe['diff'] = df_delta_pe['delta'] - .75

In [65]:
df_delta_pe.query('diff > 0').sort_values('diff')

,strike_type,open,high,low,close,volume,oi,date_only,expiry_date,bn_close,strike,days_to_expiry,side,delta,diff
date,,,,,,,,,,,,,,,
2023-09-14 09:45:00,45600PE,124.50,128.80,120.70,126.40,74595.0,589410.0,2023-09-14,2023-09-20,46096.2,45600,6,PE,0.770906,0.020906
2023-09-14 09:45:00,45500PE,103.55,107.15,100.70,105.35,143220.0,1164165.0,2023-09-14,2023-09-20,46096.2,45500,6,PE,0.801495,0.051495
2023-09-14 09:45:00,45400PE,86.80,89.70,84.60,88.30,47670.0,564255.0,2023-09-14,2023-09-20,46096.2,45400,6,PE,0.828290,0.078290
2023-09-14 09:45:00,45300PE,72.90,75.50,71.10,74.60,45975.0,477585.0,2023-09-14,2023-09-20,46096.2,45300,6,PE,0.851403,0.101403
2023-09-14 09:45:00,45200PE,61.65,63.80,59.85,62.70,31410.0,405630.0,2023-09-14,2023-09-20,46096.2,45200,6,PE,0.871989,0.121989
2023-09-14 09:45:00,45100PE,52.15,53.80,50.75,53.15,68370.0,455085.0,2023-09-14,2023-09-20,46096.2,45100,6,PE,0.889514,0.139514
2023-09-14 09:45:00,45000PE,44.95,46.45,43.95,45.80,74175.0,602355.0,2023-09-14,2023-09-20,46096.2,45000,6,PE,0.903851,0.153851
2023-09-14 09:45:00,44900PE,38.95,40.00,38.00,39.55,13230.0,230430.0,2023-09-14,2023-09-20,46096.2,44900,6,PE,0.916251,0.166251
2023-09-14 09:45:00,44800PE,33.90,34.90,33.10,34.50,35535.0,240570.0,2023-09-14,2023-09-20,46096.2,44800,6,PE,0.926627,0.176627


In [55]:
strike_ce_delta_25,strike_pe_delta_25

('46700CE', '47300PE')

In [ ]:
# 4. at every 25% we need to changes-> if any premium increasse by 25% then book the profit and 
# with the same premium amount sell 

# if strike_ce_delta_25 > ltp* .25:
#     square off
#     find strike price equal to strike price strike_pe_delta_25
#     sell it 
#     assign strike_ce_delta_25 = new_strike


# if strike_pe_delta_25 > ltp* .25:
#     square off
#     find strike price equal to strike price strike_pe_delta_25
#     sell it 
#     assign strike_ce_delta_25 = new_strike

# if strike_ce_delta_25 == strike_pe_delta:
#     print('strangle')



In [57]:
ce_delta_25_price = df_strike_day.query("strike_type == @strike_ce_delta_25")['close'].values[0]
pe_delta_25_price = df_strike_day.query("strike_type == @strike_pe_delta_25")['close'].values[0]

In [58]:
ce_delta_25_price,pe_delta_25_price

(94.15, 1231.1)

In [56]:
df_strike_day.head()

,strike_type,open,high,low,close,volume,oi,date_only,expiry_date,bn_close,strike,days_to_expiry,side
date,,,,,,,,,,,,,
2023-09-14 09:45:00,43500CE,2585.00,2585.00,2585.00,2585.00,0.0,945.0,2023-09-14,2023-09-20,46096.2,43500,6,CE
2023-09-14 09:45:00,43700PE,12.35,12.70,12.20,12.60,7710.0,90780.0,2023-09-14,2023-09-20,46096.2,43700,6,PE
2023-09-14 09:45:00,43000PE,8.40,8.55,8.35,8.55,38715.0,868590.0,2023-09-14,2023-09-20,46096.2,43000,6,PE
2023-09-14 09:45:00,47400PE,1312.30,1312.30,1312.30,1312.30,0.0,0.0,2023-09-14,2023-09-20,46096.2,47400,6,PE
2023-09-14 09:45:00,45900CE,424.00,435.75,405.30,414.00,158250.0,863040.0,2023-09-14,2023-09-20,46096.2,45900,6,CE


In [24]:

for dt_time in df_strike_day.index.unique():
        df_temp = df_strike_day.reset_index().query('date == @dt_time')

        if df_temp.query("strike_type == @strike_ce_delta_25")['close'].values[0] > ce_atm_price * 1.1:
            print("square off CE atm: ",df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0],ce_atm_price)
            p_and_l -= df_temp.query("strike_type == @ce_sell_atm")['last_price'].values[0] - ce_atm_price 
            ce_atm_flag = True

'46700CE'

'47300PE'

In [30]:
# df_delta_pe.sort_values('delta').query('delta > .25')

In [31]:
# df_temp = df_delta_ce.head(1).swifter.apply(
#     lambda x: mibian.BS(
#         [x.bn_close, x.strike, interest_rate, x.days_to_expiry], callPrice=x.close
#     ),
#     axis=1,
# )
# df_temp = pd.DataFrame(df_temp,columns= ['temp'])
# df_temp.assign(IV =lambda x: df_temp['temp'].apply(lambda cval:( cval.callDelta,cval.callPrice,cval.callTheta,cval.vega,cval.gamma) ))

In [ ]:

strike_price = 42900
position = 0
market  = False

lower_side = 42811
upper_side = 42911

target_lower = 49000
target_upper = 42488

sl_lower = 42228
sl_upper = 42299


if market:
    order_type  = kite.ORDER_TYPE_MARKET
else:
    order_type  = kite.ORDER_TYPE_LIMIT

symbol_ce = f"NFO:BANKNIFTY23504{strike_price}CE"
symbol_pe = f"NFO:BANKNIFTY23504{strike_price}PE"
# price = 385
while(True):
    bn_index = kite.ltp('NSE:NIFTY BANK')['NSE:NIFTY BANK']['last_price']
    op_begin_ce = kite.ltp(symbol_ce)[symbol_ce]['last_price']
    op_begin_pe = kite.ltp(symbol_pe)[symbol_pe]['last_price']
    print(bn_index,op_begin_ce,op_begin_pe)
    # buy 
    if bn_index > upper_side and position == 0:
        print('buy_up', bn_index) 
        trans_type = kite.TRANSACTION_TYPE_BUY
        op_begin_ce -= 5
        buy_sell_order(strike_price = strike_price,price =op_begin_ce, ce_pe = 'CE', trans_type = trans_type,order_type =order_type)
        position = 1 
        
    elif bn_index < lower_side and position == 0:
        print("buy_down",bn_index)
        trans_type = kite.TRANSACTION_TYPE_BUY
        op_begin_pe -= 5
        buy_sell_order(strike_price = strike_price,ce_pe = 'PE', trans_type = trans_type,order_type = order_type)
        position = 1
    
    # if position == 1 and (bn_index > target_upper or bn_index < sl_upper): 
    #     trans_type = kite.TRANSACTION_TYPE_SELL
    #     order_type  = kite.ORDER_TYPE_MARKET
    #     buy_sell_order(strike_price = strike_price,ce_pe = 'CE',price = None, trans_type = trans_type,order_type = order_type)
    #     break
    # elif position == 1 and (bn_index < target_lower or bn_index > sl_lower):
    #     trans_type = kite.TRANSACTION_TYPE_SELL
    #     buy_sell_order(strike_price = strike_price,ce_pe = 'PE',price = None, trans_type = trans_type,order_type = order_type)
    #     break
        
    time.sleep(5)
    clear_output(wait=True)
    # break

    